In [1]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from main import init_env_vars
from langchain import FAISS
from langchain.embeddings import OpenAIEmbeddings

init_env_vars()

db = FAISS.load_local('index', OpenAIEmbeddings())
retriever = db.as_retriever(search_type='similarity', search_kwargs={'k': 4})


def ask_bot(question, ai_instruction):
    system_template = ai_instruction + "\n\n{summaries}"
    messages = [
        SystemMessagePromptTemplate.from_template(system_template),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
    chain_type_kwargs = {"prompt": ChatPromptTemplate.from_messages(messages)}
    qa = RetrievalQAWithSourcesChain.from_chain_type(
        llm=ChatOpenAI(temperature=0),
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs=chain_type_kwargs)
    result = qa({"question": question})
    return result

In [2]:
def get_sources(result):
    source_urls = []
    for document in result['source_documents']:
        url = document.metadata['source'].replace("website/", "https://")
        if url not in source_urls:
            source_urls.append(url)

    return source_urls

In [4]:
# ai_instruction = "Give your answer in a bullet point format to be used for slide presentations"
ai_instruction = ""
question = "Can ConversationalRetrievalChain also return the source documents?"

response = ask_bot(question, ai_instruction)

print(response['answer'])
print(get_sources(response))

Yes, you can easily return source documents from the ConversationalRetrievalChain by setting the parameter `return_source_documents` to `True`. This is useful for when you want to inspect what documents were returned. Here's an example:

```
from langchain.chains import ConversationalRetrievalChain
from langchain import vectorstore
from langchain.vectorstore import as_retriever

# create a vectorstore retriever
retriever = vectorstore.as_retriever()

# create the ConversationalRetrievalChain
chain = ConversationalRetrievalChain(retriever, return_source_documents=True)

# set up some chat history and a query
chat_history = []
query = "What did the president say about Ketanji Brown Jackson"

# run the chain
result = chain({"question": query, "chat_history": chat_history})

# get the source documents
source_docs = result['source_documents']
```

In this example, `source_docs` will contain a list of the source documents that were returned by the retriever.
['https://python.langchain.com/en